# Rankings Comparison: ML vs FanGraphs vs ESPN

This notebook compares three ranking systems:
- **ML Model** (PAR-adjusted): Our machine learning predictions with positional adjustments
- **FanGraphs** (PAR-adjusted): Averaged projections from Steamer, BatX, OOPSY
- **ESPN**: ESPN's official 2026 fantasy rankings

We'll analyze correlations, identify where systems agree/disagree, and visualize the differences.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Load master rankings
df = pd.read_csv('../predictions/master_rankings_2026.csv')
print(f"Total players: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

## 1. Data Preparation

In [ ]:
# Rename columns for clarity
df = df.rename(columns={
    'ML_PAR_Rank': 'ML_Rank',
    'Proj_PAR_Rank': 'FG_Rank',
})

# Focus on the three main ranking columns
rank_cols = ['ML_Rank', 'FG_Rank', 'ESPN_Rank']

# Filter to players ranked by ESPN (not the default 300)
# This gives us a cleaner comparison set
espn_ranked = df[df['ESPN_Rank'] < 300].copy()
print(f"Players ranked by ESPN: {len(espn_ranked)}")
print(f"Players NOT ranked by ESPN: {len(df) - len(espn_ranked)}")

# Create difference columns
df['ML_vs_ESPN'] = df['ESPN_Rank'] - df['ML_Rank']  # Positive = ML ranks higher
df['FG_vs_ESPN'] = df['ESPN_Rank'] - df['FG_Rank']  # Positive = FG ranks higher
df['ML_vs_FG'] = df['FG_Rank'] - df['ML_Rank']      # Positive = ML ranks higher

espn_ranked['ML_vs_ESPN'] = espn_ranked['ESPN_Rank'] - espn_ranked['ML_Rank']
espn_ranked['FG_vs_ESPN'] = espn_ranked['ESPN_Rank'] - espn_ranked['FG_Rank']
espn_ranked['ML_vs_FG'] = espn_ranked['FG_Rank'] - espn_ranked['ML_Rank']

## 2. Overall Correlations

In [ ]:
# Correlation matrix for ESPN-ranked players
corr_matrix = espn_ranked[rank_cols].corr(method='spearman')

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0.5, 
            fmt='.3f', square=True, ax=ax,
            vmin=0, vmax=1)
ax.set_title('Spearman Rank Correlations (ESPN-Ranked Players)', fontsize=14)
plt.tight_layout()
plt.show()

print("\nSpearman correlations:")
print(corr_matrix)

In [ ]:
# Pearson correlations for comparison
corr_pearson = espn_ranked[rank_cols].corr(method='pearson')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0.5,
            fmt='.3f', square=True, ax=axes[0], vmin=0, vmax=1)
axes[0].set_title('Spearman (Rank-based)', fontsize=12)

sns.heatmap(corr_pearson, annot=True, cmap='RdYlGn', center=0.5,
            fmt='.3f', square=True, ax=axes[1], vmin=0, vmax=1)
axes[1].set_title('Pearson (Linear)', fontsize=12)

plt.suptitle('Correlation Comparison: Spearman vs Pearson', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 3. Pairwise Scatter Plots with Regression

In [ ]:
# ML vs ESPN
fig, ax = plt.subplots(figsize=(10, 8))
sns.regplot(data=espn_ranked, x='ESPN_Rank', y='ML_Rank', 
            scatter_kws={'alpha': 0.5, 's': 30},
            line_kws={'color': 'red', 'linewidth': 2},
            ax=ax)

# Add perfect agreement line
max_rank = max(espn_ranked['ESPN_Rank'].max(), espn_ranked['ML_Rank'].max())
ax.plot([0, max_rank], [0, max_rank], 'k--', alpha=0.5, label='Perfect Agreement')

# Annotate outliers (biggest disagreements)
outliers = espn_ranked[abs(espn_ranked['ML_vs_ESPN']) > 75]
for _, row in outliers.iterrows():
    ax.annotate(row['Name'], (row['ESPN_Rank'], row['ML_Rank']), 
                fontsize=8, alpha=0.7)

ax.set_xlabel('ESPN Rank', fontsize=12)
ax.set_ylabel('ML Rank (PAR-adjusted)', fontsize=12)
ax.set_title('ML Model vs ESPN Rankings', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

# Calculate correlation
r, p = stats.spearmanr(espn_ranked['ESPN_Rank'], espn_ranked['ML_Rank'])
print(f"Spearman correlation: r = {r:.3f}, p = {p:.2e}")

In [ ]:
# FanGraphs vs ESPN
fig, ax = plt.subplots(figsize=(10, 8))
sns.regplot(data=espn_ranked, x='ESPN_Rank', y='FG_Rank',
            scatter_kws={'alpha': 0.5, 's': 30},
            line_kws={'color': 'red', 'linewidth': 2},
            ax=ax)

max_rank = max(espn_ranked['ESPN_Rank'].max(), espn_ranked['FG_Rank'].max())
ax.plot([0, max_rank], [0, max_rank], 'k--', alpha=0.5, label='Perfect Agreement')

outliers = espn_ranked[abs(espn_ranked['FG_vs_ESPN']) > 75]
for _, row in outliers.iterrows():
    ax.annotate(row['Name'], (row['ESPN_Rank'], row['FG_Rank']),
                fontsize=8, alpha=0.7)

ax.set_xlabel('ESPN Rank', fontsize=12)
ax.set_ylabel('FanGraphs Rank (PAR-adjusted)', fontsize=12)
ax.set_title('FanGraphs Projections vs ESPN Rankings', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

r, p = stats.spearmanr(espn_ranked['ESPN_Rank'], espn_ranked['FG_Rank'])
print(f"Spearman correlation: r = {r:.3f}, p = {p:.2e}")

In [ ]:
# ML vs FanGraphs
fig, ax = plt.subplots(figsize=(10, 8))
sns.regplot(data=espn_ranked, x='FG_Rank', y='ML_Rank',
            scatter_kws={'alpha': 0.5, 's': 30},
            line_kws={'color': 'red', 'linewidth': 2},
            ax=ax)

max_rank = max(espn_ranked['FG_Rank'].max(), espn_ranked['ML_Rank'].max())
ax.plot([0, max_rank], [0, max_rank], 'k--', alpha=0.5, label='Perfect Agreement')

outliers = espn_ranked[abs(espn_ranked['ML_vs_FG']) > 50]
for _, row in outliers.iterrows():
    ax.annotate(row['Name'], (row['FG_Rank'], row['ML_Rank']),
                fontsize=8, alpha=0.7)

ax.set_xlabel('FanGraphs Rank (PAR-adjusted)', fontsize=12)
ax.set_ylabel('ML Rank (PAR-adjusted)', fontsize=12)
ax.set_title('ML Model vs FanGraphs Projections', fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

r, p = stats.spearmanr(espn_ranked['FG_Rank'], espn_ranked['ML_Rank'])
print(f"Spearman correlation: r = {r:.3f}, p = {p:.2e}")

## 4. Rank Difference Distributions

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# ML vs ESPN
axes[0].hist(espn_ranked['ML_vs_ESPN'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].axvline(x=espn_ranked['ML_vs_ESPN'].mean(), color='green', linestyle='-', linewidth=2, label=f"Mean: {espn_ranked['ML_vs_ESPN'].mean():.1f}")
axes[0].set_xlabel('ML Rank - ESPN Rank', fontsize=11)
axes[0].set_ylabel('Count', fontsize=11)
axes[0].set_title('ML vs ESPN\n(Positive = ML ranks higher)', fontsize=12)
axes[0].legend()

# FG vs ESPN
axes[1].hist(espn_ranked['FG_vs_ESPN'], bins=30, edgecolor='black', alpha=0.7, color='darkorange')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1].axvline(x=espn_ranked['FG_vs_ESPN'].mean(), color='green', linestyle='-', linewidth=2, label=f"Mean: {espn_ranked['FG_vs_ESPN'].mean():.1f}")
axes[1].set_xlabel('FG Rank - ESPN Rank', fontsize=11)
axes[1].set_ylabel('Count', fontsize=11)
axes[1].set_title('FanGraphs vs ESPN\n(Positive = FG ranks higher)', fontsize=12)
axes[1].legend()

# ML vs FG
axes[2].hist(espn_ranked['ML_vs_FG'], bins=30, edgecolor='black', alpha=0.7, color='forestgreen')
axes[2].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[2].axvline(x=espn_ranked['ML_vs_FG'].mean(), color='blue', linestyle='-', linewidth=2, label=f"Mean: {espn_ranked['ML_vs_FG'].mean():.1f}")
axes[2].set_xlabel('ML Rank - FG Rank', fontsize=11)
axes[2].set_ylabel('Count', fontsize=11)
axes[2].set_title('ML vs FanGraphs\n(Positive = ML ranks higher)', fontsize=12)
axes[2].legend()

plt.suptitle('Rank Difference Distributions (ESPN-Ranked Players)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Summary stats
print("\nRank Difference Summary Statistics:")
print("="*60)
for col, name in [('ML_vs_ESPN', 'ML vs ESPN'), ('FG_vs_ESPN', 'FG vs ESPN'), ('ML_vs_FG', 'ML vs FG')]:
    print(f"\n{name}:")
    print(f"  Mean: {espn_ranked[col].mean():+.1f}")
    print(f"  Median: {espn_ranked[col].median():+.1f}")
    print(f"  Std Dev: {espn_ranked[col].std():.1f}")
    print(f"  Range: [{espn_ranked[col].min():.0f}, {espn_ranked[col].max():.0f}]")

In [ ]:
# KDE plots overlaid
fig, ax = plt.subplots(figsize=(12, 6))

sns.kdeplot(data=espn_ranked['ML_vs_ESPN'], ax=ax, label='ML vs ESPN', linewidth=2)
sns.kdeplot(data=espn_ranked['FG_vs_ESPN'], ax=ax, label='FG vs ESPN', linewidth=2)
sns.kdeplot(data=espn_ranked['ML_vs_FG'], ax=ax, label='ML vs FG', linewidth=2)

ax.axvline(x=0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Rank Difference', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Rank Difference Distributions (KDE)', fontsize=14)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

## 5. Agreement Analysis

In [ ]:
# Define agreement thresholds
thresholds = [5, 10, 20, 30, 50]

agreement_stats = []
for thresh in thresholds:
    ml_espn_agree = (abs(espn_ranked['ML_vs_ESPN']) <= thresh).sum()
    fg_espn_agree = (abs(espn_ranked['FG_vs_ESPN']) <= thresh).sum()
    ml_fg_agree = (abs(espn_ranked['ML_vs_FG']) <= thresh).sum()
    
    agreement_stats.append({
        'Threshold': f'±{thresh}',
        'ML-ESPN': f"{ml_espn_agree} ({100*ml_espn_agree/len(espn_ranked):.1f}%)",
        'FG-ESPN': f"{fg_espn_agree} ({100*fg_espn_agree/len(espn_ranked):.1f}%)",
        'ML-FG': f"{ml_fg_agree} ({100*ml_fg_agree/len(espn_ranked):.1f}%)",
    })

agreement_df = pd.DataFrame(agreement_stats)
print("Players within X ranks of each other:")
print(agreement_df.to_string(index=False))

In [ ]:
# Visualize agreement rates
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(thresholds))
width = 0.25

ml_espn_pcts = [(abs(espn_ranked['ML_vs_ESPN']) <= t).mean() * 100 for t in thresholds]
fg_espn_pcts = [(abs(espn_ranked['FG_vs_ESPN']) <= t).mean() * 100 for t in thresholds]
ml_fg_pcts = [(abs(espn_ranked['ML_vs_FG']) <= t).mean() * 100 for t in thresholds]

ax.bar(x - width, ml_espn_pcts, width, label='ML vs ESPN', color='steelblue')
ax.bar(x, fg_espn_pcts, width, label='FG vs ESPN', color='darkorange')
ax.bar(x + width, ml_fg_pcts, width, label='ML vs FG', color='forestgreen')

ax.set_xlabel('Agreement Threshold (±ranks)', fontsize=12)
ax.set_ylabel('% of Players', fontsize=12)
ax.set_title('Ranking Agreement Rates', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels([f'±{t}' for t in thresholds])
ax.legend()
ax.set_ylim(0, 100)

# Add percentage labels
for i, (ml, fg, mlfg) in enumerate(zip(ml_espn_pcts, fg_espn_pcts, ml_fg_pcts)):
    ax.text(i - width, ml + 2, f'{ml:.0f}%', ha='center', fontsize=9)
    ax.text(i, fg + 2, f'{fg:.0f}%', ha='center', fontsize=9)
    ax.text(i + width, mlfg + 2, f'{mlfg:.0f}%', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Biggest Disagreements

In [ ]:
# Players where ML ranks MUCH higher than ESPN
print("=" * 80)
print("ML Model's SLEEPERS (ML ranks much higher than ESPN):")
print("=" * 80)
ml_sleepers = espn_ranked.nlargest(15, 'ML_vs_ESPN')[['Name', 'Team', 'Position', 'Type', 'ML_Rank', 'ESPN_Rank', 'ML_vs_ESPN']]
print(ml_sleepers.to_string(index=False))

In [ ]:
# Players where ESPN ranks MUCH higher than ML
print("=" * 80)
print("ML Model's FADES (ESPN ranks much higher than ML):")
print("=" * 80)
ml_fades = espn_ranked.nsmallest(15, 'ML_vs_ESPN')[['Name', 'Team', 'Position', 'Type', 'ML_Rank', 'ESPN_Rank', 'ML_vs_ESPN']]
print(ml_fades.to_string(index=False))

In [ ]:
# Players where FanGraphs ranks MUCH higher than ESPN
print("=" * 80)
print("FanGraphs SLEEPERS (FG ranks much higher than ESPN):")
print("=" * 80)
fg_sleepers = espn_ranked.nlargest(15, 'FG_vs_ESPN')[['Name', 'Team', 'Position', 'Type', 'FG_Rank', 'ESPN_Rank', 'FG_vs_ESPN']]
print(fg_sleepers.to_string(index=False))

In [ ]:
# Players where ESPN ranks MUCH higher than FanGraphs
print("=" * 80)
print("FanGraphs FADES (ESPN ranks much higher than FG):")
print("=" * 80)
fg_fades = espn_ranked.nsmallest(15, 'FG_vs_ESPN')[['Name', 'Team', 'Position', 'Type', 'FG_Rank', 'ESPN_Rank', 'FG_vs_ESPN']]
print(fg_fades.to_string(index=False))

In [ ]:
# Players where ML and FG strongly disagree
print("=" * 80)
print("ML vs FanGraphs Disagreements (ML ranks higher):")
print("=" * 80)
ml_over_fg = espn_ranked.nlargest(10, 'ML_vs_FG')[['Name', 'Team', 'Position', 'Type', 'ML_Rank', 'FG_Rank', 'ML_vs_FG']]
print(ml_over_fg.to_string(index=False))

print("\n")
print("=" * 80)
print("ML vs FanGraphs Disagreements (FG ranks higher):")
print("=" * 80)
fg_over_ml = espn_ranked.nsmallest(10, 'ML_vs_FG')[['Name', 'Team', 'Position', 'Type', 'ML_Rank', 'FG_Rank', 'ML_vs_FG']]
print(fg_over_ml.to_string(index=False))

## 7. Consensus Analysis

In [ ]:
# Find players where ALL THREE systems agree (within 20 ranks)
consensus_thresh = 20
espn_ranked['all_agree'] = (
    (abs(espn_ranked['ML_vs_ESPN']) <= consensus_thresh) & 
    (abs(espn_ranked['FG_vs_ESPN']) <= consensus_thresh) & 
    (abs(espn_ranked['ML_vs_FG']) <= consensus_thresh)
)

consensus_players = espn_ranked[espn_ranked['all_agree']].sort_values('ESPN_Rank')
print(f"Players where all 3 systems agree (within ±{consensus_thresh} ranks): {len(consensus_players)}")
print(f"That's {100*len(consensus_players)/len(espn_ranked):.1f}% of ESPN-ranked players")
print("\nTop 30 consensus players:")
print(consensus_players.head(30)[['Name', 'Team', 'Position', 'ML_Rank', 'FG_Rank', 'ESPN_Rank']].to_string(index=False))

In [ ]:
# Players where NO systems agree
no_consensus = espn_ranked[
    (abs(espn_ranked['ML_vs_ESPN']) > 30) & 
    (abs(espn_ranked['FG_vs_ESPN']) > 30)
].sort_values('ESPN_Rank')

print(f"\nPlayers with major disagreements (ML & FG both >30 from ESPN): {len(no_consensus)}")
print(no_consensus[['Name', 'Team', 'Position', 'ML_Rank', 'FG_Rank', 'ESPN_Rank', 'ML_vs_ESPN', 'FG_vs_ESPN']].head(20).to_string(index=False))

## 8. Analysis by Position/Type

In [ ]:
# Correlations by player type
print("Spearman Correlations by Player Type:")
print("=" * 60)

for ptype in ['Batter', 'SP', 'RP']:
    subset = espn_ranked[espn_ranked['Type'] == ptype]
    if len(subset) > 10:
        r_ml_espn = stats.spearmanr(subset['ML_Rank'], subset['ESPN_Rank'])[0]
        r_fg_espn = stats.spearmanr(subset['FG_Rank'], subset['ESPN_Rank'])[0]
        r_ml_fg = stats.spearmanr(subset['ML_Rank'], subset['FG_Rank'])[0]
        print(f"\n{ptype} (n={len(subset)}):")
        print(f"  ML-ESPN: {r_ml_espn:.3f}")
        print(f"  FG-ESPN: {r_fg_espn:.3f}")
        print(f"  ML-FG:   {r_ml_fg:.3f}")

In [ ]:
# Box plots of rank differences by type
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (col, title) in zip(axes, [
    ('ML_vs_ESPN', 'ML vs ESPN'),
    ('FG_vs_ESPN', 'FG vs ESPN'),
    ('ML_vs_FG', 'ML vs FG')
]):
    sns.boxplot(data=espn_ranked, x='Type', y=col, ax=ax, order=['Batter', 'SP', 'RP'])
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('')
    ax.set_ylabel('Rank Difference')

plt.suptitle('Rank Differences by Player Type', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Mean differences by type
type_summary = espn_ranked.groupby('Type').agg({
    'ML_vs_ESPN': ['mean', 'std'],
    'FG_vs_ESPN': ['mean', 'std'],
    'ML_vs_FG': ['mean', 'std'],
}).round(1)

print("\nMean Rank Differences by Player Type:")
print(type_summary)

## 9. Tier Analysis

In [ ]:
# Analyze by ESPN draft tier
espn_ranked['ESPN_Tier'] = pd.cut(
    espn_ranked['ESPN_Rank'],
    bins=[0, 25, 50, 100, 150, 200, 300],
    labels=['Top 25', '26-50', '51-100', '101-150', '151-200', '200+']
)

tier_corr = []
for tier in ['Top 25', '26-50', '51-100', '101-150', '151-200', '200+']:
    subset = espn_ranked[espn_ranked['ESPN_Tier'] == tier]
    if len(subset) > 5:
        r_ml = stats.spearmanr(subset['ML_Rank'], subset['ESPN_Rank'])[0]
        r_fg = stats.spearmanr(subset['FG_Rank'], subset['ESPN_Rank'])[0]
        tier_corr.append({'Tier': tier, 'n': len(subset), 'ML-ESPN': r_ml, 'FG-ESPN': r_fg})

tier_df = pd.DataFrame(tier_corr)
print("Correlations by ESPN Draft Tier:")
print(tier_df.to_string(index=False))

In [ ]:
# Scatter plots faceted by tier
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, tier in zip(axes, ['Top 25', '26-50', '51-100', '101-150', '151-200', '200+']):
    subset = espn_ranked[espn_ranked['ESPN_Tier'] == tier]
    if len(subset) > 0:
        ax.scatter(subset['ESPN_Rank'], subset['ML_Rank'], alpha=0.6, label='ML', s=30)
        ax.scatter(subset['ESPN_Rank'], subset['FG_Rank'], alpha=0.6, label='FG', s=30, marker='^')
        
        # Add perfect agreement line
        min_r, max_r = subset['ESPN_Rank'].min(), subset['ESPN_Rank'].max()
        ax.plot([min_r, max_r], [min_r, max_r], 'k--', alpha=0.3)
        
        ax.set_title(f'{tier} (n={len(subset)})', fontsize=11)
        ax.set_xlabel('ESPN Rank')
        ax.set_ylabel('ML/FG Rank')
        ax.legend(fontsize=9)

plt.suptitle('Rankings by ESPN Tier', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 10. Summary Statistics

In [ ]:
print("=" * 80)
print("RANKINGS COMPARISON SUMMARY")
print("=" * 80)

print(f"\nDataset: {len(espn_ranked)} ESPN-ranked players")
print(f"  Batters: {len(espn_ranked[espn_ranked['Type'] == 'Batter'])}")
print(f"  Starting Pitchers: {len(espn_ranked[espn_ranked['Type'] == 'SP'])}")
print(f"  Relief Pitchers: {len(espn_ranked[espn_ranked['Type'] == 'RP'])}")

print("\n" + "-" * 80)
print("OVERALL CORRELATIONS (Spearman):")
print("-" * 80)
print(f"  ML vs ESPN:       {stats.spearmanr(espn_ranked['ML_Rank'], espn_ranked['ESPN_Rank'])[0]:.3f}")
print(f"  FanGraphs vs ESPN: {stats.spearmanr(espn_ranked['FG_Rank'], espn_ranked['ESPN_Rank'])[0]:.3f}")
print(f"  ML vs FanGraphs:   {stats.spearmanr(espn_ranked['ML_Rank'], espn_ranked['FG_Rank'])[0]:.3f}")

print("\n" + "-" * 80)
print("AGREEMENT RATES:")
print("-" * 80)
for thresh in [10, 20, 30]:
    ml_espn = (abs(espn_ranked['ML_vs_ESPN']) <= thresh).mean() * 100
    fg_espn = (abs(espn_ranked['FG_vs_ESPN']) <= thresh).mean() * 100
    ml_fg = (abs(espn_ranked['ML_vs_FG']) <= thresh).mean() * 100
    print(f"  Within ±{thresh} ranks: ML-ESPN {ml_espn:.1f}% | FG-ESPN {fg_espn:.1f}% | ML-FG {ml_fg:.1f}%")

print("\n" + "-" * 80)
print("SYSTEMATIC BIASES (Mean Rank Differences):")
print("-" * 80)
print(f"  ML vs ESPN:  {espn_ranked['ML_vs_ESPN'].mean():+.1f} (positive = ML ranks higher)")
print(f"  FG vs ESPN:  {espn_ranked['FG_vs_ESPN'].mean():+.1f} (positive = FG ranks higher)")
print(f"  ML vs FG:    {espn_ranked['ML_vs_FG'].mean():+.1f} (positive = ML ranks higher)")

In [ ]:
# Final visualization: Joint pair plot
g = sns.pairplot(
    espn_ranked[['ML_Rank', 'FG_Rank', 'ESPN_Rank', 'Type']],
    hue='Type',
    diag_kind='kde',
    plot_kws={'alpha': 0.5, 's': 25},
    height=3
)
g.fig.suptitle('Pairwise Rank Comparisons by Player Type', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()